# Hidden Linear Function Problem

In this notebook we consider a problem from paper [1] and build a quantum cirquit, which solves it, in Cirq. 

## The problem

Consider $A \in \mathbb{F}_2^{n \times n}$ - upper-triangular binary matrix, $b \in \mathbb{F}_2^n$ - binary vector.

Define a function $q : \mathbb{F}_2^n \to \mathbb{Z}_4 $:

$$q(x) = (2 x^T A x + b^T x) ~\text{mod}~ 4, $$ 

Also define

$$\mathcal{L}_q = \Big\{x \in  \mathbb{F}_2^n : q(x \oplus y) = (q(x) + q(y)) ~\text{mod}~ 4 ~~ \forall y \in \mathbb{F}_2^n \Big\}.$$

Turns out that restriction of $q$ on $\mathcal{L}_q$ is a linear function, i.e. there exists such $z \in \mathbb{F}_2^n$, that

$$q(x) = 2 z^T x \forall x \in \mathcal{L}_q.$$

Our task is, given $A$ and $b$, to find $z$. There may be multiple answers - we need to find any such answer.

## Preparation and bruteforce solution

For small values of $n$ we can solve this problem with a trivial bruteforce solution. First, we need to build $\mathcal{L}_q$ by checking for all $2^n$ binary vectors, which of them belongs to it(by definition). Then we need to try all possible $z \in \mathbb{F}_2^n$, and for each of them and for each $x \in \mathcal{L}_q$ check whether $q(x) = 2 z^T x$.

Below we implement a class which represents instance of a problem and solves it with a bruteforce solution.

In [1]:
import numpy as np
import cirq

In [2]:
class HiddenLinearFunctionProblem:
    def __init__(self, A, b):
        self.n = A.shape[0]
        assert A.shape == (self.n, self.n)
        assert b.shape == (self.n, )
        for i in range(self.n):
            for j in range(i+1):
                assert A[i][j] == 0, 'A[i][j] can be 1 only if i<j'
        
        self.A = A
        self.b = b
        
    def q(self, x):
        assert x.shape == (self.n, )
        return (2 * (x @ self.A @ x) + (self.b @ x)) % 4
        
    def bruteforce_solve(self):
        all_vectors = [np.array([(m>>i)%2 for i in range(self.n)]) for m in range(2**self.n)]

        def vector_in_L(x):
            for y in all_vectors:
                if self.q( (x + y)%2 ) != (self.q(x) + self.q(y))%4:
                    return False
            return True

        self.L = [x for x in all_vectors if vector_in_L(x)]

        self.all_zs = [z for z in all_vectors if self.is_z(z)]
    
    # Whether given vector z is solution to this problem.
    def is_z(self, z):
        assert z.shape == (self.n, )
        assert self.L is not None
        for x in self.L:
            if self.q(x) != 2 * ((z @ x) % 2):
                return False
        return True

For testing, we need to generate an instance of a problem. We can generate random $A$ and $b$. However, for some $A$ and $b$ problem is trivial - that is, $\mathcal{L}_q = \{0\}$ and therefore any $z$ is a solution. In fact, product of $|\mathcal{L}_q|$ and number of solutions is always equal to $2^n$ (see prrof in [1]), so we want a problem with large $\mathcal{L}_q$.

Code below can be used to generate random problem with given size of $\mathcal{L}_q$.

In [3]:
def random_problem(n, seed=None):
    if seed is not None:
        np.random.seed(seed) 
    A = np.random.randint(0, 2, size=(n,n))
    for i in range(n):
        for j in range(i+1):
            A[i][j] = 0
    b = np.random.randint(0, 2, size=n)
    problem = HiddenLinearFunctionProblem(A, b)
    return problem
        
def find_interesting_problem(n, min_L_size):
    for _ in range(1000):
        problem = random_problem(n)
        problem.bruteforce_solve()
        if len(problem.L) >= min_L_size and not np.max(problem.A) == 0:
            return problem
    return None

problem = find_interesting_problem(10, 4)
print(len(problem.L), len(problem.all_zs))

4 256


We found a problem with $n=10$ and $|\mathcal{L}_q|=16$, so only 64 of 1024 possible vectors are solutions. So, chance of randomly guessing a solution is $\frac{1}{16}$.

In [4]:
A = np.array([[0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
              [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
b = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 1])
problem_10_64 = HiddenLinearFunctionProblem(A, b)
problem_10_64.bruteforce_solve()
print(len(problem_10_64.L), len(problem_10_64.all_zs))

16 64


## Solution with a quantum cirquit

As shown in [1], given problem can be solved by a quantum cirqcuit, which implements operator $H ^ {\otimes n} U_q H ^ {\otimes n}$, where

$$U_q = \prod_{1 < i < j < n} CZ_{ij}^{A_{ij}} \cdot \bigotimes_{j=1}^{n} S_j^{b_j} .$$

We need to apply this operator to $| 0^n \rangle$ and measure the result - result is guaranteed to be one of the solutions. Moreover, we can get any solution with equal probability.

Let's implement code which would generate such circuit and simulate it.

Note that: 

* We use Cirq S gate, whose matrix is $\left(\begin{smallmatrix}1 & 0\\0 & i\end{smallmatrix}\right)$. In the paper [1] matrix of S gate is defined as $\left(\begin{smallmatrix}1 & 0\\0 & -i\end{smallmatrix}\right)$. But for this problem it doesn't matter.

* We reorder CZ gates in such a way so they take less moments. This is a problem of minimal [edge coloring](https://en.wikipedia.org/wiki/Edge_coloring), and we solve it here with a simple greedy algorithm. We can do that because CZ gates commute (because their matrices are diagonal).

* All gates are Clifford gates, so we can use Clifford simulator.

In [5]:
# Given adjacency matrix A, returns list of lists of edges, such as 
# edges in each list do not have common vertex.
def graph_coloring(A):
    A = np.copy(A)
    n = A.shape[0]
    ans = []
    while np.max(A) != 0:
        edges_group = []
        used = np.zeros(n, dtype=np.bool)
        for i in range(n):
            for j in range(n):
                if A[i][j] == 1 and not used[i] and not used[j]:
                    edges_group.append((i, j))
                    A[i][j] = 0
                    used[i] = used[j] = True
        ans.append(edges_group)
    return ans
    
def generate_circuit_for_problem(problem):
    qubits = cirq.LineQubit.range(problem.n)
    circuit = cirq.Circuit()
    
    # Hadamard gates at the beginning.
    circuit += cirq.Moment([cirq.H(qubits[i]) for i in range(problem.n)])
    
    # Controlled-Z gates encoding the matrix A.
    for layer in graph_coloring(problem.A):
        for i, j in layer:
            circuit += cirq.CZ(qubits[i], qubits[j])
        
    # S gates encoding the vector b.
    S_moment = cirq.Moment()
    for i in range(problem.n):
        if problem.b[i] == 1:
            S_moment += cirq.S.on(qubits[i])
    circuit += S_moment
            
    # Hadamard gates at the end.
    circuit += cirq.Moment([cirq.H(qubits[i]) for i in range(problem.n)])
    
    # Measurements.
    circuit += cirq.Moment([cirq.measure(qubits[i], key=str(i)) for i in range(problem.n)]) 
    
    return circuit

def solve_problem(problem, print_circuit=False):
    circuit = generate_circuit_for_problem(problem)
        
    if print_circuit:
        print(circuit)
    
    sim = cirq.CliffordSimulator()
    result = sim.simulate(circuit)
    z = np.array([result.measurements[str(i)][0] for i in range(problem.n)])
    return z

solve_problem(problem_10_64, print_circuit=True)

          ┌───┐   ┌──┐   ┌───┐   ┌───┐   ┌───┐
0: ───H────@───────@──────@───────@───────@──────────────────────H───M───
           │       │      │       │       │
1: ───H────@───────┼@─────┼@──────┼@──────┼@─────@───@───@───────H───M───
                   ││     ││      ││      ││     │   │   │
2: ───H────@───────@┼─────┼┼@─────┼┼@─────┼┼─────┼───┼───┼───────H───M───
           │        │     │││     │││     ││     │   │   │
3: ───H────┼@───────@─────┼┼┼─────┼┼┼─────┼┼─────┼───┼───┼───────H───M───
           ││             │││     │││     ││     │   │   │
4: ───H────┼┼@─────@──────┼@┼─────┼┼┼─────┼┼─────┼───┼───┼───S───H───M───
           │││     │      │ │     │││     ││     │   │   │
5: ───H────┼┼@─────┼@─────@─┼─────┼@┼─────┼┼@────┼───┼───┼───S───H───M───
           ││      ││       │     │ │     │││    │   │   │
6: ───H────@┼──────┼@─────@─┼─────┼─┼─────┼@┼────┼───┼───┼───S───H───M───
            │      │      │ │     │ │     │ │    │   │   │
7: ───H────@┼──────┼@─────┼─@─────┼─┼

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 1], dtype=uint8)

Now let's test this algorithm. Let's solve it with a circuit 100 times and each time check that measurement result is indeed an answer to the problem.

In [6]:
def test_problem(problem):
    problem.bruteforce_solve()
    for _ in range(100):
        z = solve_problem(problem)
        assert problem.is_z(z)
    
test_problem(problem_10_64)
print('OK')

OK


Let's repeat that for 10 other problems with $n=8$ and chance of random guessing at most $\frac{1}{4}$.

In [7]:
for _ in range(10):
    test_problem(find_interesting_problem(8, 4))
print('OK')

OK


Now, let's run our algorithm on a problem with $n=200$.

In [8]:
%%time
problem = random_problem(200, seed=0)
solve_problem(problem, print_circuit=False)

Wall time: 9.26 s


array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1], dtype=uint8)

## Why is this problem interesting?

### 1. It's a problem without an oracle

This problem is similar to a problem solved by [Bernstein–Vazirani algorithm](https://en.wikipedia.org/wiki/Bernstein%E2%80%93Vazirani_algorithm). It also finds coefficients of unknown linear function. But in Bernstein-Vazirani algorithm this function is represented by an oracle. In this problem, the linear function is "hidden" in inputs $A$ and $b$.

### 2. Quantum circuits have advantage over classical when solving this problem

According to [Gottesman–Knill theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem), this problem can be solved in polynomial time on classical computer, because it can be solved by simulating Clifford circuit. So, it might look like quantum comuters aren't better than classical ones in solving this problem.

However, if we apply certain restrictions on matrix $A$, the circuit will have fixed depth (i.e. number of Moments). Namely, if the matrix $A$ is an adjacency matrix of a "grid" graph (whose edges can be colored in 4 colors), all CZ gates will fit in 4 moments, and overall we will have only 8 moments - and this doesn't depend on $n$.

But for classical circuits it can be proven (see [1]) that even if we restrict matrix $A$ in the same way, the depth of classical circuit (with gates of bounded fan-in) must grow as $n$ grows (in fact, it grows as $\log(n)$).

## References


[1] [Quantum advantage with shallow circuits](https://arxiv.org/pdf/1704.00690.pdf) by Sergey Bravyi, David Gosset and Robert König.